In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())
words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ","]

In [4]:
for intent in intents["intents"]:
    for pattern in intent["symptoms"]:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])
            
print(documents)

[(['fever'], 'flu'), (['cough'], 'flu'), (['sore', 'throat'], 'flu'), (['runny', 'nose'], 'flu'), (['headache'], 'flu'), (['muscle', 'aches'], 'flu'), (['fatigue'], 'flu'), (['nausea'], 'flu'), (['vomiting'], 'flu'), (['diarrhea'], 'flu'), (['abdominal', 'pain'], 'abdominal aortic aneurysm'), (['back', 'pain'], 'abdominal aortic aneurysm'), (['pain', 'in', 'the', 'groin'], 'abdominal aortic aneurysm'), (['pain', 'in', 'the', 'legs'], 'abdominal aortic aneurysm'), (['pain', 'in', 'the', 'buttocks'], 'abdominal aortic aneurysm'), (['confusion'], 'alzheimer'), (['disorientation'], 'alzheimer'), (['getting', 'lost', 'in', 'familiar', 'places'], 'alzheimer'), (['difficulty', 'planning', 'or', 'making', 'decisions'], 'alzheimer'), (['problems', 'with', 'speech', 'and', 'language'], 'alzheimer'), (['problems', 'moving', 'around', 'without', 'assistance', 'or', 'performing', 'self-care', 'tasks'], 'alzheimer'), (['personality', 'changes', ',', 'such', 'as', 'becoming', 'aggressive', ',', 'dema

In [5]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
print(words)

['a', 'abdominal', 'ache', 'aggressive', 'and', 'anormal', 'anxiety', 'around', 'assistance', 'attention', 'back', 'become', 'becoming', 'behavior', 'body', 'breath', 'buttock', 'change', 'chest', 'chill', 'completing', 'concentrating', 'confusion', 'congestion', 'controlling', 'cough', 'cracked', 'decision', 'delusion', 'demanding', 'depression', 'diarrhea', 'difficulty', 'disorientation', 'extreme', 'familiar', 'fatigue', 'feeling', 'fever', 'focusing', 'following', 'getting', 'groin', 'hallucination', 'headache', 'high', 'hyperactivity', 'hypomania', 'impulsive', 'impulsivity', 'in', 'inattentiveness', 'infected', 'inflamed', 'instruction', 'irritable', 'itchy', 'language', 'leg', 'lost', 'low', 'making', 'mania', 'may', 'mood', 'moving', 'muscle', 'nausea', 'nervous', 'nose', 'of', 'or', 'organizing', 'others', 'pain', 'part', 'paying', 'performing', 'personality', 'place', 'planning', 'problem', 'red', 'restless', 'runny', 'self-care', 'shortness', 'sitting', 'skin', 'sleep', 'sor

In [6]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [7]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [8]:
random.shuffle(training)
training = np.array(training)

X_train = list(training[:, 0])
y_train = list(training[:, 1])

C:\Users\hasan\AppData\Local\Temp/ipykernel_10080/633689806.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [9]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation="softmax"))

adam = Adam(learning_rate=0.001, decay=1e-6)

model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])
history = model.fit(np.array(X_train), np.array(y_train), 
                    epochs=100, batch_size=16, verbose=1)

Epoch 1/100
5/5 [==============================] - 3s 4ms/step - loss: 2.0816 - accuracy: 0.1471
Epoch 2/100
5/5 [==============================] - 0s 5ms/step - loss: 2.0395 - accuracy: 0.1912
Epoch 3/100
5/5 [==============================] - 0s 4ms/step - loss: 2.0598 - accuracy: 0.1176
Epoch 4/100
5/5 [==============================] - 0s 4ms/step - loss: 2.0351 - accuracy: 0.1765
Epoch 5/100
5/5 [==============================] - 0s 4ms/step - loss: 2.0104 - accuracy: 0.2941
Epoch 6/100
5/5 [==============================] - 0s 4ms/step - loss: 1.9586 - accuracy: 0.2500
Epoch 7/100
5/5 [==============================] - 0s 4ms/step - loss: 1.9525 - accuracy: 0.3824
Epoch 8/100
5/5 [==============================] - 0s 4ms/step - loss: 2.0040 - accuracy: 0.1912
Epoch 9/100
5/5 [==============================] - 0s 4ms/step - loss: 1.8814 - accuracy: 0.3529
Epoch 10/100
5/5 [==============================] - 0s 3ms/step - loss: 1.8697 - accuracy: 0.3235
Epoch 11/100
5/5 [===========

In [10]:
model.save("healthbot_model.h5", history)